In [31]:
import os
import json
import pandas as pd
from tqdm import tqdm

# 预测, 并输出成 clue 的格式

In [25]:
test_file_v10 = r"../clue_data/tnews_public/test1.0.json"
test_file_v11 = r"../clue_data/tnews_public/test.json"
label_file = r"../data/label.json"

In [26]:
test_list_v1 = []
test_list = []
label_dict ={}
with open(test_file_v10, "r", encoding="utf-8") as f:
    for line in f.readlines():
        test_list_v1.append(json.loads(line))
with open(test_file_v11, "r", encoding="utf-8") as f:
    for line in f.readlines():
        test_list.append(json.loads(line))
with open(label_file, "r", encoding="utf-8") as f:
    label_dict = json.load(f)

In [4]:
# 加载模型
import torch
import pytorch_lightning as pl
from transformers import BertTokenizer

from model import PlModel, BertLinear
from data import load_label

In [27]:
label2id, id2label = load_label(r"..\data\label.json")
model = BertLinear("bert-base-chinese", len(label2id), load_pretrain=False)
pl_model = PlModel.load_from_checkpoint("./best_model.ckpt", model=model, id2label=id2label)
pl_model.eval()
print("模型加载完成")

模型加载完成


In [29]:
pl_model.init_predict("bert-base-chinese")

In [30]:
pl_model.predict_raw(["科技创新, chaptgpt 引爆未来"])

[{'text': '科技创新, chaptgpt 引爆未来', 'pred_label': 'news_tech', 'prob': 0.5867988}]

In [32]:
# 预测, 并生成 clue 的提交文件
output_dir = "submit_clue"
os.makedirs(output_dir, exist_ok=True)

# 生成 test1.0 的提交文件
predict_file_v10 = os.path.join(output_dir, "tnews10_predict.json")
with open(predict_file_v10, "w", encoding="utf-8") as f:
    for item in tqdm(test_list_v1):
        item_id = item["id"]
        text = item["sentence"]
        label_desc = pl_model.predict_raw([text])[0]["pred_label"]
        label = label_dict[label_desc]["label"]
        f.write(json.dumps({"id": item_id, "label": label, "label_desc": label_desc}, ensure_ascii=False) + "\n")

100%|██████████| 10000/10000 [01:18<00:00, 126.89it/s]


In [33]:
# 生成 test 的提交文件
predict_file_v11 = os.path.join(output_dir, "tnews11_predict.json")
with open(predict_file_v11, "w", encoding="utf-8") as f:
    for item in tqdm(test_list):
        item_id = item["id"]
        text = item["sentence"]
        label_desc = pl_model.predict_raw([text])[0]["pred_label"]
        label = label_dict[label_desc]["label"]
        f.write(json.dumps({"id": item_id, "label": label, "label_desc": label_desc}, ensure_ascii=False) + "\n")

100%|██████████| 10000/10000 [02:34<00:00, 64.70it/s]
